<center><h2>Example 04 - Forbidding and Requiring Matches</h2></center>
<center><h2>Data taken from C.A. Floudas, Nonlinear and Mixed-Integer Optimization</h2></center>
<center><h2>Table 8.1 on page 263</h2></center></br>
We will create and solve a network as usual. We will also show how to tell ALChemE some matches cannot be done or must be done, and show how the solution changes (or not) based on our choices.

As usual, begin by importing our packages and declaring the HEN object. Note we are using a minimum $\Delta$T = 30 °C, instead of the usual 10°C.

In [ ]:
from hen_design import HEN
import unyt as u
import numpy as np
import pandas as pd

In [ ]:
myhen = HEN(delta_t = 30)

Declare the streams and utilities as usual.

In [ ]:
myhen.add_stream(95, 75, 5)
myhen.add_stream(80, 75, 50)
myhen.add_stream(30, 90, 10)
myhen.add_stream(60, 70, 12.5)

In [ ]:
myhen.add_utility('hot', 300, 0.70)
myhen.add_utility('cold', 30, 0.05)

As usual, call get_parameters().

In [ ]:
myhen.get_parameters()

<h3>Solving the system - standard</h3>
The setup is complete, so we will move on to solving the system. At first, let us solve the system without forbidding or requiring any matches.

In [ ]:
myhen.solve_HEN('above', depth = 2)

Solutions are stored in myhen.results_above or myhen.results below as pandas DataFrames. A convenient way to display all solutions is below.\
NOTE: Python numbers things from 0, so the 1st solution is solution number 0. Be careful if you are not used to this.

In [ ]:
for idx, elem in enumerate(myhen.results_above):
    print(f'Solution number {idx}:')
    print(f'Number of exchangers: {(elem.loc["Q"]>0).sum().sum():2} | Total cost: ${elem.loc["cost"].sum().sum():,.2f}')

In [ ]:
myhen.solve_HEN('below', depth = 2)

Solutions are stored in myhen.results_above or myhen.results below as pandas DataFrames. A convenient way to display all solutions is below.\
NOTE: Python numbers things from 0, so the 1st solution is solution number 0. Be careful if you are not used to this.

In [ ]:
for idx, elem in enumerate(myhen.results_below):
    print(f'Solution number {idx}:')
    print(f'Number of exchangers: {(elem.loc["Q"]>0).sum().sum():2} | Total cost: ${elem.loc["cost"].sum().sum():,.2f}')

<h3>Solving the system - forbidden matches</h3>
Let us now solve the system by forbidding a match we found in the original solution.</br>
To forbid a match, set its upper_limit = 0. First, create an array full of -1 like so:

In [ ]:
my_limit = np.full_like(myhen.upper_limit, -1) # Creating an array of -1 with the right shape

Then, set the upper heat transferred limit of whatever matches you want to block to 0

In [ ]:
my_limit[1, 1] = 0 # Forbidding the match between H1 and C1. Note the utilities come before the streams

Solving this constrained system

In [ ]:
myhen.solve_HEN('above', depth = 2, upper = my_limit)

<h3>Whoa, what happened?</h3>
ALChemE will try to obey your constraints no matter what. However, it is possible to constrain the system so much that no viable solution is found. In these scenarios, ALChemE will return an error</br>
Let us now try to forbid another stream.

In [ ]:
my_limit = np.full_like(myhen.upper_limit, -1) # Creating an array of -1 with the right shape
my_limit[1, 2] = 0 # Forbidding the match between H1 and C2. Note the utilities come before the streams

In [ ]:
myhen.solve_HEN('above', depth = 2, upper = my_limit)

<h3>Solving the system - required matches</h3>
Let us now solve the system by requiring a match we found in the original solutions.</br>
To forbid a match, set its required = 1. First, create an array full of 0 like so:

In [ ]:
my_required = np.full_like(myhen.upper_limit, 0) # Creating an array of 0 with the right shape

Then, set the required flag of whatever matches you want to block to 1

In [ ]:
my_required[1, 1] = 1 # Requiring the match between H1 and C1. Note the utilities come before the streams

In [ ]:
myhen.solve_HEN('above', depth = 2, required = my_required)

Finally, notice you can manipulate the lower limits by creating an array of 0 and changing individual values within it, then passing that array using the *lower* keyword when calling solve_HEN().\
You can also use the upper limits to limit how much is transferred in a match without forcing the limit to be 0.